In [1]:
import pandas as pd
import sqlite3

In [2]:
con = sqlite3.connect('../data/checking-logs.sqlite')

In [3]:
pd.io.sql.read_sql('select * from pageviews', con).head()

,index,uid,datetime
0,0,admin_1,2020-04-17 12:01:08.463179
1,1,admin_1,2020-04-17 12:01:23.743946
2,2,admin_3,2020-04-17 12:17:39.287778
3,3,admin_3,2020-04-17 12:17:40.001768
4,4,admin_1,2020-04-17 12:27:30.646665


In [4]:
pd.io.sql.read_sql('select * from checker', con).head(10)

,index,status,success,timestamp,numTrials,labname,uid
0,0,checking,0,2020-04-16 21:12:50.740474,5,None,admin_1
1,1,ready,0,2020-04-16 21:12:54.708365,5,code_rvw,admin_1
2,2,checking,0,2020-04-16 21:46:47.769088,7,None,admin_1
3,3,ready,0,2020-04-16 21:46:48.121217,7,lab02,admin_1
4,4,checking,0,2020-04-16 21:53:01.862637,6,code_rvw,admin_1
5,5,ready,0,2020-04-16 21:53:05.373389,6,code_rvw,admin_1
6,6,checking,0,2020-04-17 05:18:51.965864,1,None,None
7,7,ready,0,2020-04-17 05:19:02.744528,1,project1,user_4
8,8,checking,0,2020-04-17 05:22:35.249331,2,project1,user_4
9,9,ready,1,2020-04-17 05:22:45.549397,2,project1,user_4


In [5]:
datamart = pd.io.sql.read_sql('''
            SELECT checker.uid, checker.labname, 
                checker.timestamp AS  first_commit_ts, 
                pageviews.datetime AS first_view_ts
            FROM checker 
            LEFT JOIN pageviews
            ON checker.uid = pageviews.uid
            WHERE status = "ready" AND
                numTrials = 1 AND
                labname IN ('laba04', 'laba04s', 'laba05', 
                    'laba06', 'laba06s', 'project1') AND
                checker.uid LIKE 'user_%' AND
                (pageviews.datetime =
                    (SELECT MIN(pageviews.datetime)
                    FROM pageviews
                    WHERE uid = checker.uid)
                    OR pageviews.datetime IS NULL)''', con, 
                              parse_dates = ['first_commit_ts', 'first_view_ts'])

In [6]:
datamart.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140 entries, 0 to 139
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              140 non-null    object        
 1   labname          140 non-null    object        
 2   first_commit_ts  140 non-null    datetime64[ns]
 3   first_view_ts    59 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 4.5+ KB


In [7]:
datamart

,uid,labname,first_commit_ts,first_view_ts
0,user_4,project1,2020-04-17 05:19:02.744528,NaT
1,user_4,laba04,2020-04-17 11:33:17.366400,NaT
2,user_4,laba04s,2020-04-17 11:48:41.992466,NaT
3,user_17,project1,2020-04-18 07:56:45.408648,2020-04-18 10:56:55.833899
4,user_30,laba04,2020-04-18 13:36:53.971502,2020-04-17 22:46:26.785035
...,...,...,...,...
135,user_23,laba06,2020-05-21 08:34:10.517205,NaT
136,user_19,laba06s,2020-05-21 13:27:06.705881,2020-04-21 20:30:38.034966
137,user_23,laba06s,2020-05-21 14:29:15.709568,NaT
138,user_17,laba06,2020-05-21 15:21:31.567615,2020-04-18 10:56:55.833899


In [8]:
test = datamart[datamart['first_view_ts'].notna()].reset_index(drop=True)
display(test.head())
test.info()

,uid,labname,first_commit_ts,first_view_ts
0,user_17,project1,2020-04-18 07:56:45.408648,2020-04-18 10:56:55.833899
1,user_30,laba04,2020-04-18 13:36:53.971502,2020-04-17 22:46:26.785035
2,user_30,laba04s,2020-04-18 14:51:37.498399,2020-04-17 22:46:26.785035
3,user_14,laba04,2020-04-18 15:14:00.312338,2020-04-18 10:53:52.623447
4,user_14,laba04s,2020-04-18 22:30:30.247628,2020-04-18 10:53:52.623447


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59 entries, 0 to 58
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              59 non-null     object        
 1   labname          59 non-null     object        
 2   first_commit_ts  59 non-null     datetime64[ns]
 3   first_view_ts    59 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 2.0+ KB


In [9]:
control = datamart[datamart['first_view_ts'].isna()]\
                                .reset_index(drop=True)
control.fillna(test['first_view_ts'].mean(), inplace=True)
display(control.head())
control.info()

,uid,labname,first_commit_ts,first_view_ts
0,user_4,project1,2020-04-17 05:19:02.744528,2020-04-27 00:40:05.761783808
1,user_4,laba04,2020-04-17 11:33:17.366400,2020-04-27 00:40:05.761783808
2,user_4,laba04s,2020-04-17 11:48:41.992466,2020-04-27 00:40:05.761783808
3,user_2,laba04,2020-04-18 13:42:35.482008,2020-04-27 00:40:05.761783808
4,user_2,laba04s,2020-04-18 13:51:22.291271,2020-04-27 00:40:05.761783808


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81 entries, 0 to 80
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              81 non-null     object        
 1   labname          81 non-null     object        
 2   first_commit_ts  81 non-null     datetime64[ns]
 3   first_view_ts    81 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 2.7+ KB


In [10]:
test.to_sql('test', con)
control.to_sql('control', con)

In [11]:
con.close()